In [1]:
#!pip install pyfim
# !pip install pandas
# !pip install matplotlib
# !pip install unlzw
# !pip install pyfim

In [2]:
import matplotlib.pyplot as plt
from sklearn import tree
import pandas as pd
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from unlzw import unlzw
import csv, re
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score
from pyarc import TransactionDB
from pyarc.algorithms import (
    top_rules,
    createCARs,
    M1Algorithm,
    M2Algorithm
)

In [3]:
# Load the iris dataset
iris = pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/iris/iris.data', header = None)
iris = pd.read_csv('iris.data', header = None)
iris.columns=["sepallength","petalwidth","sepalwidth","petallength","class"]
iris.to_csv("iris.csv", index = None, header = True)
#iris = pd.read_csv('iris.csv',index= None, header = True)

#Load the waveform dataset #CART decision tree algorithm: 72% (Information from the .names file)
with open('waveform.data.Z', 'rb') as fh:
    compressed_data = fh.read()
    uncompressed_data = unlzw(compressed_data)
data = uncompressed_data.decode('utf-8').splitlines()
with open("wave.csv", "w") as csv_file:
    writer = csv.writer(csv_file, delimiter = '\t')
    for line in data:
        writer.writerow(re.split('\s+',line))
wave = pd.read_csv('wave.csv', header = None)
wave.to_csv("wave.csv", index = None, header = True)
wave_labels = []
for i in range(21):
  wave_labels.append('x'+str(i))
wave_labels.append('class')
wave.columns=wave_labels
wave.to_csv("wave.csv", index = None, header = True)

#Load the breast cancer dataset
breastCancer = pd.read_csv("breast-cancer.data")
breastCancer.columns=["class","age", "menopause", "tumor-size", "inv-nodes", "node-caps","deg-malig","breast","breast-quad","irradiat"]
breastCancer.to_csv("breastCancer.csv", index = None, header = True)
#wave.to_csv("breastCancer.csv", index = None, header = True)


#Load the german dataset
german = pd.read_csv('german.data', header = None)
german.head()
german_columns=["checkAccStatus", "durationMth", "credHist", "purpose", "credAmt","savAccBond","emplySince","instRate","personalStatSex","otherDebtGuar","presResSince","prpty","age(years)","otherInstallPlans","housing","numExistCreds","job","numPplMaintain","telephone","frgnWorker","goodBad"]
german = german[0].str.split(" ", n = 20, expand = True)
german.columns = german_columns
german.to_csv("german.csv", index = None, header = True)
#german.columns = german_columns

#Load the heart dataset
heart = pd.read_csv('heart.dat', header = None)
heart = heart[0].str.split(" ", n = 13, expand = True)
heart.columns=["age", "sex", "chest pain type", "resting blood pressure", "serum cholesterol (mg/dl)", "resting blood sugar >120mg/dl","resting electrocariographic results","maximum heart rate received","exercise induced angina", "oldpeak","slopePeak", "numMajorVessels","thal","class"]
heart_columns= ["age", "sex", "chest pain type", "resting blood pressure", "serum cholesterol (mg/dl)", "resting blood sugar >120mg/dl","resting electrocariographic results","maximum heart rate received","exercise induced angina", "oldpeak","slopePeak", "numMajorVessels","thal"]
heart.to_csv("heart.csv", index = None, header = True)


#Load the kaggle dataset
#stroke
stroke = pd.read_csv("healthcare-dataset-stroke-data.csv", sep=";", index_col=0)

#Heart Attack
campusPlacement = pd.read_csv("Placement_Data_Full_Class.csv", sep=";", index_col=0)

In [4]:
print(wave_labels)

['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20', 'class']


Discretising the rules

In [5]:
import Orange

In [6]:
from yarc import CBA
from yarc.Structure import TransactionDB
from yarc.Mine_Classi_Alg.generating_CARS import ClassAssocationRule, Antecedent, Consequent, top_rules, CARlist
from yarc.Mine_Classi_Alg.m2classi import M2Classi
from yarc.Mine_Classi_Alg.predictor import Predictor
import pandas as pd

ModuleNotFoundError: No module named 'yarc'

In [ ]:
def discretiseRule(X):
    temp = Orange.data.Table(X)
    disc = Orange.preprocess.Discretize()
    disc.method = Orange.preprocess.discretize.EqualFreq(n=3)
    d_temp = disc(temp)
    X= table_to_frame(d_temp)
    return X

In [ ]:
def runCBA(X,y,target):
    X = discretiseRule(X)

    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=14, test_size=0.2, stratify = y)#25
    train = pd.concat([X_train, y_train], axis=1)
    test=pd.concat([X_test, y_test], axis=1)
    txns = TransactionDB.from_DataFrame(train, target=target) #../../
    txnstest = TransactionDB.from_DataFrame(test, target=target) #../../

    cba = CBA()
    cba.fit(txns)
    cba.target_class
    cba.pre.rules
    
    cbaTrainAcc=cba.rule_model_accuracy(txns)
    cbaTestAcc=cba.rule_model_accuracy(txnstest)
    # get the best association rules
    rules = top_rules(txns.string_representation)

    # convert them to class association rules
    cars = CARlist(rules)

    predictor = M2Classi(cars, txns).build()

    accuracy = predictor.test_transactions(txnstest)
    #print(cba.pre.rules)
    print("CBA Train Accuracy:",str(cbaTrainAcc))
    print("CBA Test Accuracy:",str(cbaTestAcc))
    print("Top K Test Accuracy:",str(accuracy))

In [ ]:
from sklearn.model_selection import train_test_split
import pandas as pd
from Orange.data.pandas_compat import table_from_frame,table_to_frame

read_file = pd.read_csv ('iris.csv') #discretised iris from their database #currently its the pure one without discretising

X=read_file[["sepallength","petalwidth","sepalwidth","petallength"]]
y=read_file[["class"]]

X = discretiseRule(X)

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=14, test_size=0.2, stratify = y)
train = pd.concat([X_train, y_train], axis=1)
test=pd.concat([X_test, y_test], axis=1)
txns = TransactionDB.from_DataFrame(train, target="class") #../../
txnstest = TransactionDB.from_DataFrame(test, target="class") #../../

iris_cba = CBA()

In [ ]:
iris_cba.fit(txns)
iris_cba.target_class
iris_cba.predict_probability(txns)
iris_cba.pre.rules
ClassAssocationRule(Antecedent({}), Consequent("class", "default_class1"), 0.1, 0.2)
iris_cba.predict_matched_rules(txns)

In [ ]:
cbaTrainAcc=iris_cba.rule_model_accuracy(txns)
cbaTestAcc=iris_cba.rule_model_accuracy(txnstest)
# get the best association rules
rules = top_rules(txns.string_representation)

# convert them to class association rules
cars = CARlist(rules)

predictor = M2Classi(cars, txns).build()


accuracy = predictor.test_transactions(txnstest)
print("CBA Train Accuracy:",str(cbaTrainAcc))
print("CBA Test Accuracy:",str(cbaTestAcc))
print("Top K Test Accuracy:",str(accuracy))

In [ ]:
read_file = pd.read_csv ('wave.csv')
X=read_file[['x0', 'x1', 'x2', 'x3', 'x4', 'x5', 'x6', 'x7', 'x8', 'x9', 'x10', 'x11', 'x12', 'x13', 'x14', 'x15', 'x16', 'x17', 'x18', 'x19', 'x20']]
y=read_file[["class"]]
runCBA(X,y,"class")

Heart

In [ ]:
read_file = pd.read_csv ('heart.csv')
X=read_file[["age", "sex", "chest pain type", "resting blood pressure", "serum cholesterol (mg/dl)", "resting blood sugar >120mg/dl","resting electrocariographic results","maximum heart rate received","exercise induced angina", "oldpeak","slopePeak", "numMajorVessels","thal"]]
y=read_file[["class"]]

runCBA(X,y,"class")

Breast Cancer

In [ ]:
#error: cannot convert string to float
read_file = pd.read_csv ('breastCancer.csv')
X=read_file[["age", "menopause", "tumor-size", "inv-nodes", "node-caps","deg-malig","breast","breast-quad","irradiat"]]
y=read_file[["class"]]
runCBA(X,y,"class")

In [ ]:
#error: cannot convert string to float
read_file = pd.read_csv ('breastCancer.csv')
X=read_file[["age", "menopause", "tumor-size", "inv-nodes", "node-caps","deg-malig","breast","breast-quad","irradiat"]]
y=read_file[["class"]]
runCBA(X,y,"class")

German

In [ ]:
#error: cannot convert string to float
read_file = pd.read_csv ('german.csv')
X=read_file[["checkAccStatus", "durationMth", "credHist", "purpose", "credAmt","savAccBond","emplySince","instRate","personalStatSex","otherDebtGuar","presResSince","prpty","age(years)","otherInstallPlans","housing","numExistCreds","job","numPplMaintain","telephone","frgnWorker"]]
y=read_file[["goodBad"]]
runCBA(X,y,"goodBad")

In [ ]:
cba.predict_probability(txns)

In [ ]:
ClassAssocationRule(Antecedent({}), Consequent("class", "default_class1"), 0.1, 0.2)

In [ ]:
cba.rule_model_accuracy(txns)

In [ ]:
cba.rule_model_accuracy(txnstest)

In [ ]:
# get the best association rules
rules = top_rules(txns.string_representation)

# convert them to class association rules
cars = CARlist(rules)
predictor = M2Classi(cars, txns).build()


accuracy = predictor.test_transactions(txnstest)


In [ ]:
print(accuracy)

In [ ]:
read_file = pd.read_csv ('heart.csv')
X=read_file[ ["age", "sex", "chest pain type", "resting blood pressure", "serum cholesterol (mg/dl)", "resting blood sugar >120mg/dl","resting electrocariographic results","maximum heart rate received","exercise induced angina", "oldpeak","slopePeak", "numMajorVessels","thal"]]
y=read_file[["class"]]
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=14, test_size=0.2, stratify = y)#25
train = pd.concat([X_train, y_train], axis=1)
test=pd.concat([X_test, y_test], axis=1)
txns = TransactionDB.from_DataFrame(train, target="class") #../../
txnstest = TransactionDB.from_DataFrame(test, target="class") #../../

cba = CBA()

In [ ]:
cba.fit(txns)
cba.target_class
cba.pre.rules

In [ ]:
cba.predict_probability(txns)

In [ ]:
ClassAssocationRule(Antecedent({}), Consequent("class", "default_class1"), 0.1, 0.2)

In [ ]:
cba.rule_model_accuracy(txns)

In [ ]:
cba.rule_model_accuracy(txnstest)

In [ ]:
# get the best association rules
rules = top_rules(txns.string_representation)

# convert them to class association rules
cars = CARlist(rules)

predictor = M2Classi(cars, txns).build()


accuracy = predictor.test_transactions(txnstest)
print(accuracy)

In [ ]:
pip install pyarc